# Livrable final du projet - Projet Green Graph
*Equipe CesiCDP - Chef de projet : Leila | Opérateurs : Tom, Edwin*</br>
## Sommaire :
    1. Introduction

    PARTIE 1 : Modélisation
    2. Rappel de la modélisation formelle
        2.1. Résumé des hypothèses et représentations
        2.2. Justification du maintien de la modélisation
    3. Méthodes de résolution
        3.1. Choix des algorithmes
        3.2. Description des algorithmes 
        3.3. Complexité des algorithmes 

    PARTIE 2 : Implémentation et exploitation
    4. Implémentation
        4.1. Détails de l’implémentation des algorithmes
        4.2. Cas de test représentatifs
        4.3. Résultats observés sur les tests
    5. Étude expérimentale
        5.1. Méthodologie du plan d’expérience
        5.2. Analyse des performances 
        5.3. Limites observées
        5.4. Perspectives d’amélioration

    6. Conclusion

    7. Annexes
        7.1. Glossaire
        7.2. Références bibliographiques
        7.3. Annexes techniques 